In [14]:
pd.to_datetime('2021-01-02') + pd.tseries.offsets.MonthBegin(-2)

Timestamp('2020-12-01 00:00:00')

In [49]:
%%backtest
# from datetime import datetime

start = '2020-01-01'
end = '2023-02-28'
universe = StockUniverse('ZZ800')
# 沪深300：由上海和深圳证券市场中市值大、流动性好的300只股票组成，综合反映中国A股市场上市股票价格的整体表现。
# 中证500：由全部A股中剔除沪深300指数成份股及总市值排名前300名的股票后，总市值排名靠前的500只股票组成，综合反映中国A股市场中一批中小市值公司的股票价格表现。
# 中证800：中证800指数由中证500和沪深300指数成份股组成，综合反映中国A股市场大中小市值公司的股票价格表现。
benchmark = 'HS300'
freq = 'd'
refresh_rate = Monthly(-1)

accounts = {
    'stock_account': AccountConfig(account_type='security', capital_base=1e6)
}


def initialize(context):
    pass


def handle_data(context):
    current_universe = context.get_universe(asset_type='stock',exclude_halt=True)
    today = context.current_date
#     print(today)
    if today.strftime('%d') == '01':
        prev_month_begin = today + pd.tseries.offsets.MonthBegin(-1)
    else:
        prev_month_begin = today + pd.tseries.offsets.MonthBegin(-2)
#     print('prev_month_begin:', prev_month_begin)
    try:
        factor_exposure = get_data_cube(symbol=current_universe,
                                        field=['d6dk9218qq.size', 'd6dk9218qq.rev','d6dk9218qq.bm',
                                               'd6dk9218qq.illiq', 'd6dk9218qq.ivol'],
                                        start=prev_month_begin, end=today,
                                        style='ast').to_frame().reset_index()
    except AttributeError:
        factor_exposure = pd.DataFrame()
#     print(factor_exposure.columns)
    if factor_exposure.shape[0] == 0:
        pass
    else:
        factor_exposure.rename(columns={'major':'date','minor':'sec_id',
                                        'd6dk9218qq.size':'size',
                                        'd6dk9218qq.bm':'bm',
                                        'd6dk9218qq.rev':'rev',
                                        'd6dk9218qq.illiq':'illiq',
                                        'd6dk9218qq.ivol':'ivol'},inplace=True)
#         print(factor_exposure)
#         print(factor_exposure['date'].unique()[-1])
#         factor_exposure = factor_exposure[factor_exposure['date'] == factor_exposure['date'].unique()[-1]]
        ##### Intersection #####
#         stocks_set = {}
#         stocks_set['rev'] = set(factor_exposure.sort_values('rev', ascending=True)['sec_id'].iloc[0:200].tolist())
#         stocks_set['size'] = set(factor_exposure.sort_values('size', ascending=True)['sec_id'].iloc[0:200].tolist())
#         stocks_set['illiq'] = set(factor_exposure.sort_values('illiq', ascending=False)['sec_id'].iloc[0:200].tolist())
#         stocks_set['ivol'] = set(factor_exposure.sort_values('ivol', ascending=True)['sec_id'].iloc[0:200].tolist())
#         stocks_set['bm'] = set(factor_exposure.sort_values('bm', ascending=False)['sec_id'].iloc[0:200].tolist())
#         buy_list = set.intersection(
#                                     stocks_set['rev'],
#                                     stocks_set['size'],
#                                     stocks_set['illiq'],
#                                     stocks_set['ivol'],
#                                     stock_set['bm']
#         )
#         buy_list = list(buy_list)
        ##### Scoring #####
        factor_exposure['rev_rank'] = factor_exposure['rev'].rank()
        factor_exposure['size_rank'] = factor_exposure['size'].rank()
        factor_exposure['illiq_rank'] = factor_exposure['illiq'].rank(ascending=False)
        factor_exposure['ivol_rank'] = factor_exposure['ivol'].rank()
        factor_exposure['bm_rank'] = factor_exposure['bm'].rank(ascending=False)
        factor_exposure['rank_sum'] = factor_exposure['illiq_rank'] + \
                                      factor_exposure['rev_rank'] + \
                                        factor_exposure['ivol_rank'] + \
                                      factor_exposure['size_rank'] + \
                                     factor_exposure['bm_rank']
        factor_exposure.sort_values('rank_sum', inplace=True)
        factor_exposure.reset_index(drop=True,inplace=True)
#         print(factor_exposure)
        nstocks = factor_exposure.shape[0]
        buy_list = list(factor_exposure['sec_id'].iloc[0:50])

        stock_account = context.get_account('stock_account')
        current_positions = stock_account.get_positions(exclude_halt=True)

        for stock in current_positions:
            if stock not in buy_list:
                stock_account.order_to(stock, 0)
        if len(buy_list) == 0:
            pass
        else:
            for stock in buy_list:
                stock_account.order_pct_to(stock, 1/len(buy_list))


Chart()

In [47]:
# #查看调仓记录
show_order(start,end)

Order()

In [48]:
# #查看持仓记录
show_position(start,end)

Position()